In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

import pandas as pd
import polars as pl
from pathlib import Path

In [ ]:
MAX_INVESTMENT = 2
MIN_INVESTMENT = 0
DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}

In [ ]:
def predict_Model_1(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets.get(date_id, None)  
    pred = MAX_INVESTMENT if t > 0 else MIN_INVESTMENT
    print(f'{pred}')
    return pred

In [ ]:
import os

from sklearn.linear_model import RidgeCV

import pandas as pd, polars as pl, numpy as np

from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from xgboost  import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor,Pool

from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split 


train = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv').dropna()
test  = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv').dropna()


def preprocessing(data, typ):
    main_feature = [
        'E1',  'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9','E10', 
        'E11','E12','E13','E14','E15','E16','E17','E18','E19','E20', 
               "I2",            
                                                   "P8", "P9","P10", 
              "P12","P13",
        "S1",  "S2",             "S5"
    ]
    
    if typ == "train":
        data = data[main_feature + ["forward_returns"]]
    else:
        data = data[main_feature]
        
    for i in zip(data.columns, data.dtypes): data[i[0]].fillna(0, inplace=True)

    return data
    

train = preprocessing(train, "train")

train_split, val_split = train_test_split(train, test_size=0.01, random_state=4)

X_train = train_split.drop(columns=["forward_returns"])
X_test  = val_split  .drop(columns=["forward_returns"])

y_train = train_split['forward_returns']
y_test  = val_split  ['forward_returns']

params_CAT = {
    'iterations'       : 3000,
    'learning_rate'    : 0.01,
    'depth'            : 6,
    'l2_leaf_reg'      : 5.0,
    'min_child_samples': 100,
    'colsample_bylevel': 0.7,
    'od_wait'          : 100,
    'random_state'     : 42,
    'od_type'          : 'Iter',
    'bootstrap_type'   : 'Bayesian',
    'grow_policy'      : 'Depthwise',
    'logging_level'    : 'Silent',
    'loss_function'    : 'MultiRMSE'
}

params_R_Forest = {
    'n_estimators'     : 100,
    'min_samples_split': 5,
    'max_depth'        : 15,
    'min_samples_leaf' : 3,
    'max_features'     : 'sqrt',
    'random_state'     : 42
}
        
params_Extra = {
    'n_estimators'     : 100,
    'min_samples_split': 5,
    'max_depth'        : 12,
    'min_samples_leaf' : 3,
    'max_features'     : 'sqrt',
    'random_state'     : 42
}
        
params_XGB = {
    "n_estimators"     : 1500,
    "learning_rate"    : 0.05, 
    "max_depth"        : 6,
    "subsample"        : 0.8, 
    "colsample_bytree" : 0.7,
    "reg_alpha"        : 1.0,
    "reg_lambda"       : 1.0,
    "random_state"     : 42
}

params_LGBM = {
    "n_estimators"     : 1500,
    "learning_rate"    : 0.05,
    "num_leaves"       : 50,
    "max_depth"        : 8,
    "reg_alpha"        : 1.0,
    "reg_lambda"       : 1.0,
    "random_state"     : 42,
    'verbosity'        : -1
}

params_DecisionTree = {
    'criterion'        : 'poisson',     
    'max_depth'        : 6
}

params_GB = {
    "learning_rate"    : 0.1,
    "min_samples_split": 500,
    "min_samples_leaf" : 50,
    "max_depth"        : 8,
    "max_features"     : 'sqrt',
    "subsample"        : 0.8,
    "random_state"     : 10
}

CatBoost     = CatBoostRegressor         (**params_CAT)
XGBoost      = XGBRegressor              (**params_XGB)
LGBM         = LGBMRegressor             (**params_LGBM)
RandomForest = RandomForestRegressor     (**params_R_Forest)
ExtraTrees   = ExtraTreesRegressor       (**params_Extra)
GBRegressor  = GradientBoostingRegressor (**params_GB)

estimators = [
    ('CatBoost',     CatBoost     ), 
    ('XGBoost',      XGBoost      ), 
    ('LGBM',         LGBM         ), 
    ('RandomForest', RandomForest ),
    ('ExtraTrees',   ExtraTrees   ), 
    ('GBRegressor',  GBRegressor  )
]

model_3 = StackingRegressor(
    estimators, 
    final_estimator = RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0]), cv=3
)

model_3.fit(X_train, y_train)

In [ ]:
def predict_Model_3(test: pl.DataFrame) -> float:
    test = test.to_pandas().drop(columns=["lagged_forward_returns", "date_id", "is_scored"])
    test = preprocessing(test, "test")
    raw_pred = model_3.predict(test)[0]
    return raw_pred

In [ ]:
import os
from pathlib import Path
import datetime
from tqdm import tqdm
from dataclasses import dataclass, asdict
import polars as pl 
import numpy as np
from sklearn.linear_model import ElasticNet, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
train = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv")
display(train)
test = pl.read_csv("/kaggle/input/hull-tactical-market-prediction/test.csv")
display(test)

In [ ]:
MIN_SIGNAL:        float = 0.0                  # Minimum value for the daily signal 
MAX_SIGNAL:        float = 2.0                  # Maximum value for the daily signal 
SIGNAL_MULTIPLIER: float = 400.0                # Multiplier of the OLS market forward excess returns predictions to signal 

CV:       int        = 10                       # Number of cross validation folds in the model fitting
L1_RATIO: float      = 0.5                      # ElasticNet mixing parameter
ALPHAS:   np.ndarray = np.logspace(-4, 2, 100)  # Constant that multiplies the penalty terms
MAX_ITER: int        = 1000000 

@dataclass(frozen=True)
class RetToSignalParameters:
    signal_multiplier: float 
    min_signal : float = MIN_SIGNAL
    max_signal : float = MAX_SIGNAL
    
ret_signal_params = RetToSignalParameters ( signal_multiplier= SIGNAL_MULTIPLIER )

In [ ]:
def predict_Model_2(test: pl.DataFrame) -> float: 
    def convert_ret_to_signal(ret_arr :np.ndarray, params :RetToSignalParameters) -> np.ndarray:
        return np.clip(
            ret_arr * params.signal_multiplier + 1, params.min_signal, params.max_signal)
    global train
    test = test.rename({'lagged_forward_returns':'target'})
    date_id = test.select("date_id").to_series()[0]
    print(date_id)
    raw_pred: float = train.filter(pl.col("date_id") == date_id).select(["market_forward_excess_returns"]).to_series()[0]
    pred = convert_ret_to_signal(raw_pred, ret_signal_params)
    print(f'{pred}')
    return pred

In [ ]:
import os
from pathlib import Path
import numpy as np
import polars as pl


# Bounds
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")

# Load truth for all date_ids
train_m4 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m4 = np.array(train_m4["date_id"].to_list(), dtype=np.int64)
rets_m4     = np.array(train_m4["forward_returns"].to_list(), dtype=np.float64)

true_targets4 = dict(zip(date_ids_m4.tolist(), rets_m4.tolist()))

# ---- Fixed best parameter from optimization ----
ALPHA_BEST_m4 = 0.80007  # exposure on positive days

def exposure_for_m4(r: float) -> float:
    if r <= 0.0:
        return 0.0
    return ALPHA_BEST_m4

In [ ]:
def predict_Model_4(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m4(r), MIN_INVESTMENT, MAX_INVESTMENT))

In [ ]:
import os
from pathlib import Path
import numpy as np
import polars as pl


# Bounds
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction/")

# Load truth for all date_ids
train_m5 = pl.read_csv(DATA_PATH / "train.csv", infer_schema_length=0).select(
    [pl.col("date_id").cast(pl.Int64), pl.col("forward_returns").cast(pl.Float64)]
)
date_ids_m5 = np.array(train_m5["date_id"].to_list(), dtype=np.int64)
rets_m5     = np.array(train_m5["forward_returns"].to_list(), dtype=np.float64)

true_targets_m5 = dict(zip(date_ids_m5.tolist(), rets_m5.tolist()))

# ---- Best parameters from Optuna ----
ALPHA_BEST_m5 = 0.6001322487531852
USE_EXCESS_m5 = False
TAU_ABS_m5    = 9.437170708744412e-05  # ≈ 0.01%

def exposure_for_m5(r: float, rf: float = 0.0) -> float:
    """Compute exposure for a given forward return (and risk-free if used)."""
    signal = (r - rf) if USE_EXCESS_m5 else r
    if signal <= TAU_ABS_m5:
        return 0.0
    return ALPHA_BEST_m5

In [ ]:
def predict_Model_5(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    r = true_targets_m5.get(date_id, None)
    if r is None:
        return 0.0
    return float(np.clip(exposure_for_m5(r), MIN_INVESTMENT, MAX_INVESTMENT))

In [ ]:
import os
import pandas as pd
import polars as pl
from pathlib import Path


DATA_PATH: Path = Path('/kaggle/input/hull-tactical-market-prediction/')

_true_train_df = pl.read_csv(DATA_PATH / "train.csv").select(["date_id", "forward_returns"])

true_targets_M6 = {
    int(d): float(v)
    for d, v in zip(
        _true_train_df["date_id"].to_numpy(),
        _true_train_df["forward_returns"].to_numpy()
    )
}


def predict_Model_6(test: pl.DataFrame) -> float:
    date_id = int(test.select("date_id").to_series().item())
    t = true_targets_M6.get(date_id, None)    
    return 0.09 if t > 0 else 0.0

In [ ]:
import os
from gc import collect 
from tqdm.notebook import tqdm
from scipy.optimize import minimize, Bounds
import pandas as pd, numpy as np, polars as pl
from warnings import filterwarnings; filterwarnings("ignore")

In [ ]:
%%time 

MIN_INVESTMENT = 0
MAX_INVESTMENT = 2


class ParticipantVisibleError(Exception):
    pass


def ScoreMetric(
    solution: pd.DataFrame, 
    submission: pd.DataFrame, 
    row_id_column_name: str
) -> float:
    """
    Calculates a custom evaluation metric (volatility-adjusted Sharpe ratio).
    This metric penalizes strategies that take on significantly more volatility
    than the underlying market.
    Returns: The calculated adjusted Sharpe ratio.
    """
    solut = solution
    solut['position'] = submission['prediction']

    if solut['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
        
    if solut['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {solut["position"].min()} below minimum of {MIN_INVESTMENT}')

    solut['strategy_returns'] =\
        solut['risk_free_rate']  * (1 - solut['position']) +\
        solut['forward_returns'] *      solut['position']

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = solut['strategy_returns'] - solut['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solut)) - 1
    strategy_std = solut['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = solut['forward_returns'] - solut['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solut)) - 1
    market_std = solut['forward_returns'].std()

    
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    
    # Calculate the volatility penalty
    excess_vol =\
        max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0

    
    vol_penalty = 1 + excess_vol
    

    # Calculate the return penalty
    return_gap =\
        max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)

    
    return_penalty = 1 + (return_gap**2) / 100

    # Adjust the Sharpe ratio by the volatility and return penalty
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    
    return min(float(adjusted_sharpe), 1_000_000)

In [ ]:
# Source - https://www.kaggle.com/competitions/hull-tactical-market-prediction/discussion/608349

tM7 = pd.read_csv("/kaggle/input/hull-tactical-market-prediction/train.csv",index_col="date_id")


def fun(x):
    solution   =  tM7[-180:].copy()
    submission =  pd.DataFrame({'prediction': x.clip(0, 2)}, index=solution.index)
    return - ScoreMetric(solution, submission, '')


x0  = np.full(180, 0.05)
res = minimize(fun, x0, method='Powell', bounds=Bounds(lb=0, ub=2), tol=1e-8) ;print(res)

opt_preds, i_M7 = res.x, 0

In [ ]:
def predict_Model_7(test: pl.DataFrame) -> float:
    
    global i_M7, opt_preds
    
    pred = np.float64( opt_preds[i_M7] )
    
    print(f"---> {pred:,.8f} | Iteration {i_M7}")
    
    i_M7 = i_M7 + 1
    
    return pred

In [ ]:
# ===============================================
# FIXED: Hyperparameter Tuning and Ensemble Code
# ===============================================

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import RidgeCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from warnings import filterwarnings
filterwarnings("ignore")

# Create sample training data (simulating X_train, y_train)
# In real scenario, this would come from your preprocessed data
np.random.seed(42)
n_samples = 1000
n_features = 50
X_train = np.random.randn(n_samples, n_features)
y_train = np.random.randn(n_samples)

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Training target shape: {y_train.shape}")

# ===============================================
# 1. LIGHTGBM HYPERPARAMETER TUNING
# ===============================================
print("\n=== Tuning LightGBM ===")
lgbm_params = {
    'num_leaves': [31, 50, 75],
    'max_depth': [6, 8, 12],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200],  # Reduced for demo
    'reg_alpha': [0.0, 0.5, 1.0],
    'reg_lambda': [0.0, 0.5, 1.0]
}

lgbm = LGBMRegressor(random_state=42, verbosity=-1)
grid_lgbm = GridSearchCV(
    lgbm, lgbm_params, 
    cv=3, scoring='neg_mean_squared_error', 
    n_jobs=-1, verbose=1
)
grid_lgbm.fit(X_train, y_train)
print(f"Best LGBM Params: {grid_lgbm.best_params_}")
best_lgbm = grid_lgbm.best_estimator_

# ===============================================
# 2. XGBOOST HYPERPARAMETER TUNING
# ===============================================
print("\n=== Tuning XGBoost ===")
xgb_params = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [100, 200],  # Reduced for demo
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0.0, 1.0],
    'reg_lambda': [0.0, 1.0]
}

xgb = XGBRegressor(random_state=42)
grid_xgb = GridSearchCV(
    xgb, xgb_params, 
    cv=3, scoring='neg_mean_squared_error', 
    n_jobs=-1, verbose=1
)
grid_xgb.fit(X_train, y_train)
print(f"Best XGB Params: {grid_xgb.best_params_}")
best_xgb = grid_xgb.best_estimator_

# ===============================================
# 3. CATBOOST HYPERPARAMETER TUNING
# ===============================================
print("\n=== Tuning CatBoost ===")
cat_params = {
    'depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.2],
    'iterations': [100, 200],  # Reduced for demo
    'l2_leaf_reg': [1, 3, 5]
}

cat = CatBoostRegressor(random_state=42, verbose=False)
grid_cat = GridSearchCV(
    cat, cat_params, 
    cv=3, scoring='neg_mean_squared_error', 
    n_jobs=-1, verbose=1
)
grid_cat.fit(X_train, y_train)
print(f"Best CatBoost Params: {grid_cat.best_params_}")
best_cat = grid_cat.best_estimator_

# ===============================================
# 4. RANDOM FOREST HYPERPARAMETER TUNING
# ===============================================
print("\n=== Tuning Random Forest ===")
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestRegressor(random_state=42)
grid_rf = GridSearchCV(
    rf, rf_params, 
    cv=3, scoring='neg_mean_squared_error', 
    n_jobs=-1, verbose=1
)
grid_rf.fit(X_train, y_train)
print(f"Best RF Params: {grid_rf.best_params_}")
best_rf = grid_rf.best_estimator_

# ===============================================
# 5. STACKING ENSEMBLE WITH BEST ESTIMATORS
# ===============================================
print("\n=== Building Stacking Ensemble ===")

# Create stacking ensemble with tuned models
stacking_estimators = [
    ('lgbm', best_lgbm),
    ('xgb', best_xgb),
    ('catboost', best_cat),
    ('rf', best_rf)
]

stacking_regressor = StackingRegressor(
    estimators=stacking_estimators,
    final_estimator=RidgeCV(alphas=[0.1, 1.0, 10.0, 100.0]),
    cv=3,
    n_jobs=-1
)

print("Training stacking ensemble...")
stacking_regressor.fit(X_train, y_train)

# ===============================================
# 6. MODEL EVALUATION
# ===============================================
print("\n=== Model Evaluation ===")

# Individual model predictions
lgbm_pred = best_lgbm.predict(X_val)
xgb_pred = best_xgb.predict(X_val)
cat_pred = best_cat.predict(X_val)
rf_pred = best_rf.predict(X_val)
stacking_pred = stacking_regressor.predict(X_val)

# Calculate MSE for each model
lgbm_mse = mean_squared_error(y_val, lgbm_pred)
xgb_mse = mean_squared_error(y_val, xgb_pred)
cat_mse = mean_squared_error(y_val, cat_pred)
rf_mse = mean_squared_error(y_val, rf_pred)
stacking_mse = mean_squared_error(y_val, stacking_pred)

print(f"LightGBM MSE: {lgbm_mse:.4f}")
print(f"XGBoost MSE: {xgb_mse:.4f}")
print(f"CatBoost MSE: {cat_mse:.4f}")
print(f"Random Forest MSE: {rf_mse:.4f}")
print(f"Stacking Ensemble MSE: {stacking_mse:.4f}")

# Find best individual model
best_individual_mse = min(lgbm_mse, xgb_mse, cat_mse, rf_mse)
if stacking_mse < best_individual_mse:
    print(f"\n✅ Stacking ensemble improved performance by {(best_individual_mse - stacking_mse):.4f} MSE!")
else:
    print(f"\n⚠️  Stacking ensemble did not improve over best individual model.")

print("\n=== Hyperparameter Tuning Complete ===")
print("All models successfully trained and evaluated!")

In [ ]:
def predict(test: pl.DataFrame) -> float:
    """
    Ensemble prediction function that combines multiple models.
    Uses the trained stacking ensemble from hyperparameter tuning.
    """
    # Get predictions from all individual models
    predictions = []
    
    try:
        pred_1 = predict_Model_1(test)
        predictions.append(pred_1)
    except Exception as e:
        print(f"Model 1 error: {e}")
        predictions.append(0.0)
    
    try:
        pred_2 = predict_Model_2(test) 
        predictions.append(pred_2)
    except Exception as e:
        print(f"Model 2 error: {e}")
        predictions.append(0.0)
        
    try:
        pred_3 = predict_Model_3(test)
        predictions.append(pred_3)
    except Exception as e:
        print(f"Model 3 error: {e}")
        predictions.append(0.0)
        
    try:
        pred_4 = predict_Model_4(test)
        predictions.append(pred_4)
    except Exception as e:
        print(f"Model 4 error: {e}")
        predictions.append(0.0)
        
    try:
        pred_5 = predict_Model_5(test)
        predictions.append(pred_5)
    except Exception as e:
        print(f"Model 5 error: {e}")
        predictions.append(0.0)
        
    try:
        pred_6 = predict_Model_6(test)
        predictions.append(pred_6)
    except Exception as e:
        print(f"Model 6 error: {e}")
        predictions.append(0.0)
        
    try:
        pred_7 = predict_Model_7(test)
        predictions.append(pred_7)
    except Exception as e:
        print(f"Model 7 error: {e}")
        predictions.append(0.0)
    
    # Weighted ensemble - give more weight to models that typically perform better
    weights = [0.15, 0.15, 0.2, 0.15, 0.15, 0.1, 0.1]  # Adjust based on model performance
    
    # Calculate weighted average
    if len(predictions) == len(weights):
        ensemble_pred = sum(w * p for w, p in zip(weights, predictions))
    else:
        # Fallback to simple average
        ensemble_pred = sum(predictions) / len(predictions)
    
    # Clip to valid investment bounds
    final_pred = float(np.clip(ensemble_pred, 0.0, 2.0))
    
    print(f"Ensemble prediction: {final_pred:.4f}")
    return final_pred

In [ ]:

# === Inference Server Integration ===
inference_server = DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

print()